In [1]:
import glob
from PIL import Image, ImageOps
from src.atom_seg import *
import cv2

In [2]:
main_image = '0034302.jpg' #rebar
root_path = '/home/wenisch/Atom360/AI/Learning/data/dataset/'
# Label ids of the dataset
category_ids = {
    # "outlier": 0,
    "rebar": 1,
    "spall": 2,
    "crack": 3,
}

In [3]:
image_id = 0
annotations = []
images = []
main_img = Image.open(root_path+"images/"+ main_image)
width, height = main_img.size
# print("main_img.size", main_img.size)
# The mask image is *.png but the original image is *.jpg.
# We make a reference to the original file in the COCO JSON file
filename = os.path.basename(main_image).split(".")[0]
image = create_image_annotation(filename, width, height, image_id)
images.append(image)
rebarmask_file = os.path.join(root_path, "masks",filename + 'rebar' + '.jpg')
spallmask_file = os.path.join(root_path, "masks", filename + 'spall' + '.jpg')
crackmask_file = os.path.join(root_path, "masks", filename + 'crack' + '.jpg')

# rebarmask = np.zeros_like(main_img)
# spallmask = np.zeros_like(main_img)
# crackmask = np.zeros_like(main_img)
# Open the image and (to be sure) we convert it to RGB
thresh = 150
fn = lambda x : 255 if x > thresh else 0
# if os.path.exists(rebarmask_file):
mask1 = cv2.imread(rebarmask_file)
Conv_hsv_Gray = cv2.cvtColor(mask1, cv2.COLOR_BGR2GRAY)
ret,rebarmask = cv2.threshold(Conv_hsv_Gray,127,255,cv2.THRESH_BINARY)
category_id = category_ids["rebar"]
annotations.append(create_annotation_format2(rebarmask, category_id, image_id))
# if os.path.exists(spallmask_file):
#     mask2 = cv2.imread(spallmask_file)#.convert("1")
#     Conv_hsv_Gray = cv2.cvtColor(mask2, cv2.COLOR_BGR2GRAY)
#     ret,spallmask = cv2.threshold(Conv_hsv_Gray,127,255,cv2.THRESH_BINARY)
#     category_id = category_ids["spall"]
#     annotations.append(create_annotation_format2(spallmask, category_id, image_id))
# if os.path.exists(crackmask_file):
#     mask3 = cv2.imread(crackmask_file)#.convert("1")
#     Conv_hsv_Gray = cv2.cvtColor(mask3, cv2.COLOR_BGR2GRAY)
#     ret,crackmask = cv2.threshold(Conv_hsv_Gray,127,255,cv2.THRESH_BINARY)
#     category_id = category_ids["crack"]
#     annotations.append(create_annotation_format2(crackmask, category_id, image_id))

In [11]:
annotations[0]['bbox']

[array([  0., 201., 337., 268.]),
 array([216.,  61., 324., 220.]),
 array([  3.,   0., 100.,  63.])]

In [13]:
annotations[0]['bbox'][0].tolist()

[0.0, 201.0, 337.0, 268.0]

In [ ]:
ground_truth_binary_mask= cv2.copyMakeBorder(rebarmask,1,1,1,1,cv2.BORDER_CONSTANT,value=black)
fortran_ground_truth_binary_mask = np.asfortranarray(ground_truth_binary_mask)
encoded_ground_truth = mask.encode(fortran_ground_truth_binary_mask)
ground_truth_area = mask.area(encoded_ground_truth)
ground_truth_bounding_box = mask.toBbox(encoded_ground_truth)
contours = measure.find_contours(ground_truth_binary_mask, 0.5)
bbox = ground_truth_bounding_box.tolist()
bbox

In [ ]:
type(encoded_ground_truth)

In [ ]:
def encode_mask_to_poly(mask, mask_id, image_id):
    if len(mask.shape) == 3:
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

    kernel = np.ones((2, 2), np.uint8)

    mask = cv2.dilate(mask, kernel, iterations=1)
    _, C, h = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    seg = [[float(x) for x in contour.flatten()] for contour in C]
    seg = [cont for cont in seg if len(cont) > 4]  # filter all polygons that are boxes
    rle = mask.frPyObjects(seg, mask.shape[0], mask.shape[1])

    return {
        'area': float(sum(mask.area(rle))),
        'bbox': list(mask.toBbox(rle)[0]),
        'category_id': 1,
        'id': mask_id,
        'image_id': image_id,
        'iscrowd': 0,
        'segmentation': seg
    }

In [ ]:
# ground_truth_binary_mask.shape
# fortran_ground_truth_binary_mask.shape


In [ ]:
len(contours)